# 3D Visualization of SA Optimized UAV Routes

This notebook visualizes the 3D lattice graph and the optimized drone routes from the Simulated Annealing algorithm using interactive Plotly graphics similar to D_graph_test.ipynb.

In [ ]:
# Import required libraries
import numpy as np
import networkx as nx
from geopy.distance import geodesic
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import random
import math
import time
from datetime import datetime
from collections import defaultdict
from dataclasses import dataclass
from typing import List, Tuple, Dict, Set, Optional
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducible results
random.seed(39)
np.random.seed(39)

In [ ]:
# Import SA classes and functions from the main script
# Note: You may need to adjust the import path based on your file structure
import sys
sys.path.append('.')

# Import all the classes from SA_Convergence_Test.py
exec(open('SA_Convergence_Test.py').read())

In [ ]:
def create_3d_graph_visualization(graph, temporal_trajectories=None, title="3D UAV Graph and Routes"):
    """
    Create 3D visualization of the graph and drone routes using Plotly.
    
    Args:
        graph: NetworkX graph with spatial structure
        temporal_trajectories: List of TemporalTrajectory objects
        title: Plot title
    """
    
    # Get node positions
    pos = nx.get_node_attributes(graph, 'pos')
    availability = nx.get_node_attributes(graph, 'available')
    
    # Create lists for nodes
    node_x = [pos[node][1] for node in graph.nodes() if node in pos]  # Longitude
    node_y = [pos[node][0] for node in graph.nodes() if node in pos]  # Latitude
    node_z = [pos[node][2] for node in graph.nodes() if node in pos]  # Altitude
    node_colors = ['green' if availability.get(node, True) else 'red' for node in graph.nodes() if node in pos]
    node_text = [f"Node: {node}<br>Lat: {pos[node][0]:.6f}<br>Lon: {pos[node][1]:.6f}<br>Alt: {pos[node][2]:.1f}m" 
                 for node in graph.nodes() if node in pos]
    
    # Create lists for graph edges (background network)
    edge_x, edge_y, edge_z = [], [], []
    
    for edge in graph.edges():
        if edge[0] in pos and edge[1] in pos:
            x0, y0, z0 = pos[edge[0]][1], pos[edge[0]][0], pos[edge[0]][2]
            x1, y1, z1 = pos[edge[1]][1], pos[edge[1]][0], pos[edge[1]][2]
            
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
            edge_z.extend([z0, z1, None])
    
    # Create the main graph edge trace (background network)
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        line=dict(width=1, color='lightgray'),
        hoverinfo='none',
        mode='lines',
        name='Graph Edges',
        opacity=0.3
    )
    
    # Create the node trace
    node_trace = go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers',
        hoverinfo='text',
        marker=dict(size=3, color=node_colors, opacity=0.6),
        text=node_text,
        name='Graph Nodes'
    )
    
    # Initialize traces list
    traces = [edge_trace, node_trace]
    
    # Add drone route traces if provided
    if temporal_trajectories:
        # Define colors for different drones
        drone_colors = ['red', 'blue', 'orange', 'purple', 'brown', 'pink', 'cyan', 'magenta']
        
        for i, trajectory in enumerate(temporal_trajectories):
            drone_color = drone_colors[i % len(drone_colors)]
            
            # Create route path coordinates
            route_x, route_y, route_z = [], [], []
            route_text = []
            
            for j, (node, time_point) in enumerate(zip(trajectory.spatial_path, trajectory.time_points)):
                if node in pos:
                    route_x.append(pos[node][1])  # Longitude
                    route_y.append(pos[node][0])  # Latitude
                    route_z.append(pos[node][2])  # Altitude
                    
                    speed = trajectory.speeds[j] if j < len(trajectory.speeds) else 0.0
                    route_text.append(
                        f"Drone {i}<br>" +
                        f"Node: {node}<br>" +
                        f"Waypoint: {j+1}/{len(trajectory.spatial_path)}<br>" +
                        f"Time: {time_point:.2f}s<br>" +
                        f"Speed: {speed:.1f} m/s<br>" +
                        f"Lat: {pos[node][0]:.6f}<br>" +
                        f"Lon: {pos[node][1]:.6f}<br>" +
                        f"Alt: {pos[node][2]:.1f}m"
                    )
            
            # Create route line trace
            route_trace = go.Scatter3d(
                x=route_x, y=route_y, z=route_z,
                mode='lines+markers',
                line=dict(width=8, color=drone_color),
                marker=dict(size=6, color=drone_color, symbol='circle'),
                hoverinfo='text',
                text=route_text,
                name=f'Drone {i} Route (ID: {trajectory.drone_id})',
                opacity=0.8
            )
            traces.append(route_trace)
            
            # Add start and end markers
            if route_x:  # Ensure route is not empty
                # Start marker
                start_trace = go.Scatter3d(
                    x=[route_x[0]], y=[route_y[0]], z=[route_z[0]],
                    mode='markers',
                    marker=dict(size=12, color='green', symbol='diamond'),
                    hoverinfo='text',
                    text=[f"START - Drone {i}<br>Node: {trajectory.spatial_path[0]}<br>Time: {trajectory.time_points[0]:.2f}s"],
                    name=f'Drone {i} Start',
                    showlegend=False
                )
                traces.append(start_trace)
                
                # End marker
                end_trace = go.Scatter3d(
                    x=[route_x[-1]], y=[route_y[-1]], z=[route_z[-1]],
                    mode='markers',
                    marker=dict(size=12, color='red', symbol='x'),
                    hoverinfo='text',
                    text=[f"END - Drone {i}<br>Node: {trajectory.spatial_path[-1]}<br>Time: {trajectory.time_points[-1]:.2f}s"],
                    name=f'Drone {i} End',
                    showlegend=False
                )
                traces.append(end_trace)
    
    # Create the figure
    fig = go.Figure(data=traces)
    
    # Customize the layout
    fig.update_layout(
        title={
            'text': title,
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 20}
        },
        scene=dict(
            xaxis_title='Longitude',
            yaxis_title='Latitude',
            zaxis_title='Altitude (m)',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.2)
            ),
            aspectmode='cube'
        ),
        width=1200,
        height=800,
        showlegend=True,
        legend=dict(
            x=0.02,
            y=0.98,
            bgcolor='rgba(255,255,255,0.8)'
        )
    )
    
    return fig

In [ ]:
def create_temporal_animation(graph, temporal_trajectories, time_step=1.0):
    """
    Create an animated visualization showing drone movement over time.
    
    Args:
        graph: NetworkX graph
        temporal_trajectories: List of TemporalTrajectory objects
        time_step: Time step for animation frames (seconds)
    """
    
    if not temporal_trajectories:
        print("No trajectories provided for animation")
        return None
    
    # Get time range
    min_time = min(traj.time_points[0] for traj in temporal_trajectories)
    max_time = max(traj.time_points[-1] for traj in temporal_trajectories)
    
    # Create time steps
    time_steps = np.arange(min_time, max_time + time_step, time_step)
    
    pos = nx.get_node_attributes(graph, 'pos')
    
    # Create frames for animation
    frames = []
    drone_colors = ['red', 'blue', 'orange', 'purple', 'brown', 'pink', 'cyan', 'magenta']
    
    for t in time_steps:
        frame_traces = []
        
        # Add background graph (static)
        edge_x, edge_y, edge_z = [], [], []
        for edge in list(graph.edges())[:500]:  # Limit edges for performance
            if edge[0] in pos and edge[1] in pos:
                x0, y0, z0 = pos[edge[0]][1], pos[edge[0]][0], pos[edge[0]][2]
                x1, y1, z1 = pos[edge[1]][1], pos[edge[1]][0], pos[edge[1]][2]
                edge_x.extend([x0, x1, None])
                edge_y.extend([y0, y1, None])
                edge_z.extend([z0, z1, None])
        
        frame_traces.append(go.Scatter3d(
            x=edge_x, y=edge_y, z=edge_z,
            line=dict(width=1, color='lightgray'),
            hoverinfo='none',
            mode='lines',
            opacity=0.2,
            showlegend=False
        ))
        
        # Add drone positions at current time
        for i, trajectory in enumerate(temporal_trajectories):
            drone_color = drone_colors[i % len(drone_colors)]
            
            # Find drone position at time t
            current_pos = trajectory.get_position_at_time(t)
            
            if current_pos and current_pos in pos:
                x, y, z = pos[current_pos][1], pos[current_pos][0], pos[current_pos][2]
                
                # Add current drone position
                frame_traces.append(go.Scatter3d(
                    x=[x], y=[y], z=[z],
                    mode='markers',
                    marker=dict(size=15, color=drone_color, symbol='circle'),
                    hoverinfo='text',
                    text=[f"Drone {i}<br>Time: {t:.1f}s<br>Node: {current_pos}<br>Lat: {pos[current_pos][0]:.6f}<br>Lon: {pos[current_pos][1]:.6f}<br>Alt: {pos[current_pos][2]:.1f}m"],
                    name=f'Drone {i}',
                    showlegend=(t == time_steps[0])  # Only show legend for first frame
                ))
                
                # Add trail (path traveled so far)
                trail_x, trail_y, trail_z = [], [], []
                for j, (node, time_point) in enumerate(zip(trajectory.spatial_path, trajectory.time_points)):
                    if time_point <= t and node in pos:
                        trail_x.append(pos[node][1])
                        trail_y.append(pos[node][0])
                        trail_z.append(pos[node][2])
                
                if len(trail_x) > 1:
                    frame_traces.append(go.Scatter3d(
                        x=trail_x, y=trail_y, z=trail_z,
                        mode='lines',
                        line=dict(width=4, color=drone_color),
                        hoverinfo='none',
                        opacity=0.6,
                        showlegend=False
                    ))
        
        frames.append(go.Frame(data=frame_traces, name=f"t={t:.1f}"))
    
    # Create initial figure
    fig = go.Figure(data=frames[0].data, frames=frames)
    
    # Add animation controls
    fig.update_layout(
        title="Animated UAV Route Execution",
        scene=dict(
            xaxis_title='Longitude',
            yaxis_title='Latitude',
            zaxis_title='Altitude (m)',
            aspectmode='cube'
        ),
        updatemenus=[{
            'type': 'buttons',
            'buttons': [
                {
                    'label': 'Play',
                    'method': 'animate',
                    'args': [None, {'frame': {'duration': 200, 'redraw': True}, 'fromcurrent': True}]
                },
                {
                    'label': 'Pause',
                    'method': 'animate',
                    'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate', 'transition': {'duration': 0}}]
                }
            ],
            'x': 0.1,
            'y': 0
        }],
        sliders=[{
            'steps': [
                {
                    'args': [[f"t={t:.1f}"], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                    'label': f"{t:.1f}s",
                    'method': 'animate'
                } for t in time_steps
            ],
            'x': 0.1,
            'len': 0.8,
            'xanchor': 'left',
            'y': 0,
            'yanchor': 'top',
            'transition': {'duration': 0},
            'currentvalue': {'prefix': 'Time: ', 'visible': True, 'xanchor': 'right'}
        }]
    )
    
    return fig

In [ ]:
# Run the SA optimization and get results
print("Running Simulated Annealing Optimization...")

# Configuration
NUM_DRONES = 6
GRAPH_DIMENSIONS = (15, 15, 3)  # Reduced for better visualization
TIME_STEP_DURATION = 1.0

# Build graph
builder = UAVGraphBuilder(n_lat=GRAPH_DIMENSIONS[0], 
                         n_lon=GRAPH_DIMENSIONS[1], 
                         n_alt=GRAPH_DIMENSIONS[2])
graph = builder.build_graph()

# Generate routes
routes = generate_uav_routes(graph, NUM_DRONES)

print(f"Generated {len(routes)} routes")
print(f"Graph has {len(graph.nodes())} nodes and {len(graph.edges())} edges")

# Initialize optimizer
optimizer = SimulatedAnnealingOptimizer(graph, TIME_STEP_DURATION)

# Run optimization (with reduced parameters for faster execution)
best_trajectories, final_cost, tracker = optimizer.simulated_annealing_with_tracking(
    routes, 
    initial_temp=100,     # Lower temp for faster convergence
    cooling_rate=0.95,    # Faster cooling
    min_temp=1.0          # Higher min temp
)

print(f"\nOptimization complete!")
print(f"Final cost: {final_cost:.2f}")
print(f"Number of trajectories: {len(best_trajectories) if best_trajectories else 0}")

In [ ]:
# Create 3D static visualization
if best_trajectories:
    print("Creating 3D visualization...")
    
    fig_3d = create_3d_graph_visualization(
        graph, 
        best_trajectories, 
        title=f"3D UAV Route Visualization - {len(best_trajectories)} Drones (SA Optimized)"
    )
    
    fig_3d.show()
    
    # Save the plot
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"3d_uav_routes_{len(best_trajectories)}drones_{timestamp}.html"
    fig_3d.write_html(filename)
    print(f"3D visualization saved to {filename}")
else:
    print("No trajectories to visualize")

In [ ]:
# Create animated visualization (optional - may be resource intensive)
CREATE_ANIMATION = False  # Set to True if you want to create animation

if CREATE_ANIMATION and best_trajectories:
    print("Creating animated visualization...")
    print("Note: This may take some time and resources")
    
    fig_animated = create_temporal_animation(graph, best_trajectories, time_step=2.0)
    
    if fig_animated:
        fig_animated.show()
        
        # Save animation
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"animated_uav_routes_{len(best_trajectories)}drones_{timestamp}.html"
        fig_animated.write_html(filename)
        print(f"Animated visualization saved to {filename}")
else:
    print("Animation creation skipped (set CREATE_ANIMATION = True to enable)")

In [ ]:
# Display route statistics
if best_trajectories:
    print("\n" + "="*60)
    print("ROUTE VISUALIZATION SUMMARY")
    print("="*60)
    
    for i, trajectory in enumerate(best_trajectories):
        print(f"\n🚁 Drone {i} (ID: {trajectory.drone_id}):")
        print(f"   Route: {trajectory.spatial_path[0]} → {trajectory.spatial_path[-1]}")
        print(f"   Waypoints: {len(trajectory.spatial_path)}")
        print(f"   Duration: {trajectory.get_duration():.2f} seconds")
        print(f"   Distance: {trajectory.get_total_distance():.2f} meters")
        if trajectory.speeds:
            print(f"   Avg Speed: {np.mean(trajectory.speeds):.2f} m/s")
    
    print(f"\n📊 Fleet Summary:")
    total_distance = sum(traj.get_total_distance() for traj in best_trajectories)
    max_time = max(traj.get_duration() for traj in best_trajectories)
    print(f"   Total Distance: {total_distance:.2f} meters")
    print(f"   Fleet Makespan: {max_time:.2f} seconds")
    print(f"   Average Route Length: {total_distance/len(best_trajectories):.2f} meters")
    
    # Validate solution
    conflicts = optimizer.event_validator.validate_solution(best_trajectories, graph)
    print(f"   Conflicts: {len(conflicts)} {'✅' if len(conflicts) == 0 else '⚠️'}")
    
    print("="*60)